In [116]:
from pydantic import BaseModel
from typing import List, Dict, Any
import yaml
import yaml
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any
from enum import Enum
import importlib

# from pydantic import Field, BaseModel, Optional
from typing import Optional, Dict, Any, List

class LLMConfig(BaseModel):
    type: str  # LLM type (e.g., "openai", "azure")
    model_kwargs: Dict[str, Any]  # Model-specific arguments

class GenerationOptionsConfig(BaseModel):
    llms: List[LLMConfig]
    eval_data_set_path: str
    prompt_template_path: Optional[str] = None 
    read_local_only: bool
    evaluator: str
    retriever: Optional[str] = None 

    @classmethod
    def from_yaml(cls, file_path: str) -> "GenerationOptionsConfig":
        with open(file_path, "r") as yaml_file:
            config = yaml.safe_load(yaml_file)
        return cls(**config["Generation"])
    
class GenerationConfig(BaseModel):
    type: str  # Specifies the LLM type (e.g., "openai", "azure")
    model_kwargs: Optional[Dict[str, Any]] = Field(default_factory=dict, description="Model-specific parameters")

# Example usage
if __name__ == "__main__":
    config_path = "config.yaml"
    generation_options_config = GenerationOptionsConfig.from_yaml(config_path)
    print(generation_options_config)


llms=[LLMConfig(type='openai', model_kwargs={'model_name': 'gpt-4', 'temperature': 0.7}), LLMConfig(type='azure_openai', model_kwargs={'model_name': 'azure-gpt-4', 'temperature': 0.6})] eval_data_set_path='path/to/eval_data.csv' prompt_template_path='path/to/prompt_templates.yaml' read_local_only=True evaluator='custom_evaluator' retriever='custom_retriever'


/Users/ashwinaravind/.pyenv/versions/3.12.5/lib/python3.12/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_kwargs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [96]:
import yaml
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any
from enum import Enum
import importlib

# Step 1: Lazy Loading Helper Function
def lazy_load(module_name: str, class_name: str):
    try:
        # Dynamically import the module
        module = importlib.import_module(module_name)
        # Get the class from the module
        return getattr(module, class_name)
    except Exception as e:
        raise ValueError(f"Error loading {class_name} from module {module_name}: {e}")

# Step 2: Enum Class for LLM Types
class LLM(str, Enum):
    OPENAI = "openai"
    AZURE_OPENAI = "azure_openai"
    HUGGINGFACE = "huggingface"
    OLLAMA = "ollama"
    COHERE = "cohere"
    VERTEXAI = "vertexai"
    BEDROCK = "bedrock"
    JINA = "jina"
    CUSTOM = "custom"

# Step 3: Map LLM Types to Lazy-loaded Embedding Classes
LLM_MAP = {
    LLM.OPENAI: lazy_load("langchain_openai", "ChatOpenAI"),
    LLM.AZURE_OPENAI: lazy_load("langchain_openai", "AzureChatOpenAI"),
}

# Step 4: Define the LLM Configuration Model
class LLMConfig(BaseModel):
    model_config = {"protected_namespaces": ()}
    
    type: LLM  # Enum to specify the LLM
    model_kwargs: Optional[Dict[str, Any]] = Field(default_factory=dict, description="Model-specific parameters like model name/type")
    custom_class: Optional[str] = None  # Optional: If using a custom class

# Step 5: Load Configuration from YAML
def load_config_from_yaml(file_path: str) -> LLMConfig:
    print("read yaml")
    with open(file_path, "r") as file:
        config_data = yaml.safe_load(file)
    # Convert to LLMConfig object
    return LLMConfig(**config_data["llm"])



In [97]:
import yaml
import os
import requests
from ragbuilder.generation.config import PromptTemplate
import pandas as pd
def load_prompts(file_name: str = "rag_prompts.yaml", url: str= os.getenv("RAG_PROMPT_URL"),read_local: bool = False):
    """
    Load YAML prompts either from a local file or an online source.

    Args:
        file_name (str): Name of the YAML file. Defaults to "rag_prompts.yaml".
        read_local (bool): If True, read from a local file. Otherwise, fetch from an online URL.

    Returns:
        List[PromptTemplate]: A list of PromptTemplate objects.
    """
    yaml_content = None

    if read_local:
        # Attempt to read from the local file
        if os.path.exists(file_name):
            print(f"Loading prompts from local file: {file_name}")
            with open(file_name, 'r') as f:
                yaml_content = f.read()
        else:
            raise FileNotFoundError(f"Local file not found: {file_name}")
    else:
        # Attempt to fetch from an online source
        print(f"Fetching prompts from online file: {url}")
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an HTTP error for bad responses
            yaml_content = response.text
        except requests.exceptions.RequestException as e:
            raise RuntimeError(f"Failed to load prompts from URL {url}: {e}")

    # Parse the YAML content
    try:
        prompts_data = yaml.safe_load(yaml_content)
    except yaml.YAMLError as e:
        raise ValueError(f"Failed to parse YAML content: {e}")

    # Convert YAML entries into PromptTemplate objects
    prompts = [
        PromptTemplate(name=entry['name'], template=entry['template'])
        for entry in prompts_data
    ]
    return prompts

In [112]:
from typing import List, Dict, Type

class SystemPromptGenerator:
    def __init__(self, config: "GenerationOptionsConfig", evaluator_class: Type):
        """
        Initialize the SystemPromptGenerator with generation options and evaluator class.

        Args:
            config (GenerationOptionsConfig): Configuration object containing options for generation.
            evaluator_class (Type): Evaluator class to be instantiated for evaluation.
        """
        self.config = config
        self.evaluator = evaluator_class()
        self.prompt_templates = load_prompts(config.prompt_template_path)
        print(self.config)

    # @staticmethod
    # def _load_prompt_templates(file_path: str) -> list:
    #     """
    #     Load prompt templates from a YAML file.

    #     Args:
    #         file_path (str): Path to the YAML file containing prompt templates.

    #     Returns:
    #         list: A list of loaded prompt templates.
    #     """
    #     try:
    #         with open(file_path, "r") as yaml_file:
    #             templates = yaml.safe_load(yaml_file)
    #         return templates.get("prompts", [])
    #     except Exception as e:
    #         raise ValueError(f"Failed to load prompt templates: {e}")

    def _build_trial_config(self, options_config: "GenerationOptionsConfig") -> List[GenerationConfig]:
        """
        Build a list of GenerationConfig objects from the provided GenerationOptionsConfig.

        Args:
            options_config (GenerationOptionsConfig): The input configuration for trial generation.

        Returns:
            List[GenerationConfig]: A list of generated configurations for trials.
        """
        trial_configs = []
        for llm_config in options_config.llms:
            llm_instance = LLMConfig(type=llm_config.type, model_kwargs=llm_config.model_kwargs)
            print(llm_instance)
            trial_config = GenerationConfig(
                type=llm_instance,  # Pass the LLMConfig instance here
                model_kwargs=llm_config.model_kwargs,
                evaluator=options_config.evaluator,
                retriever=options_config.retriever,
                eval_data_set_path=options_config.eval_data_set_path,
                prompt_template_path=options_config.prompt_template_path,
                read_local_only=options_config.read_local_only,)
            trial_configs.append(trial_config)
        trial_configs = []
        # for llm_config in options_config.llms:
        #     print(llm_config)
        #     trial_config = GenerationConfig(
        #         llm_type=llm_config.type,
        #         llm_model_kwargs=llm_config.model_kwargs,
        #         evaluator=options_config.evaluator,
        #         retriever=options_config.retriever,
        #         eval_data_set_path=options_config.eval_data_set_path,
        #         prompt_template_path=options_config.prompt_template_path,
        #         read_local_only=options_config.read_local_only,
        #     )
        #     trial_configs.append(trial_config)
        return trial_configs
        return None

    def optimize(self):
        """
        Optimize the prompt templates by running the pipeline for each generation configuration
        and evaluating the results.

        Returns:
            dict: A dictionary containing the best prompt template and its score.
        """
        print("Optimization started...")

        # Generate all trial configurations.
        trial_configs = self._build_trial_config(self.config)
        return 
        best_prompt = None
        best_score = float("-inf")
        for trial_config in trial_configs:
            print(f"Running pipeline for LLM: {trial_config.llm_model_kwargs['model']}...")

            # Call the pipeline function with the trial configuration.
            pipeline_results = self._run_pipeline(trial_config)
            print(f"Pipeline results: {pipeline_results}")
            # Evaluate the results using the evaluator.
            # evaluated_results = self.evaluator.evaluate(
            #     pipeline_results,
            #     llm=trial_config.llm_model_kwargs,
            #     embeddings=trial_config.llm_model_kwargs,  # Adjust this based on embedding logic.
            # )

            # # Compute the average score for this trial.
            # average_score = self._calculate_average_score(evaluated_results)
            # print(f"Average Score for {trial_config.llm_model_kwargs['model']}: {average_score}")

            # # Update the best prompt template if this trial is better.
            # if average_score > best_score:
            #     best_score = average_score
            #     best_prompt = trial_config

        print(f"Optimization completed. Best Score: {best_score}")
        return {
            "best_prompt": 'best_prompt',
            "best_score": 'best_score',
        }

    def _run_pipeline(self, trial_config: GenerationConfig) -> List[Dict]:
        """
        Placeholder function to run the RAG pipeline for the given trial configuration.

        Args:
            trial_config (GenerationConfig): The configuration to use for the pipeline.

        Returns:
            List[Dict]: Results from running the pipeline.
        """
        # Replace with your actual pipeline invocation logic.
        # For now, this is a placeholder.
        return [
            {"question": "What is AI?", "answer": "Artificial Intelligence", "context": "AI is ..."}
        ]

    def _calculate_average_score(self, evaluated_results: List[Dict]) -> float:
        """
        Calculate the average score from evaluated results.

        Args:
            evaluated_results (List[Dict]): The results from the evaluator.

        Returns:
            float: The average score across all results.
        """
        scores = [result.get("score", 0) for result in evaluated_results]
        return sum(scores) / len(scores) if scores else 0.0


In [113]:

from abc import ABC, abstractmethod
from datasets import Dataset
import pandas as pd
from ragbuilder.generation.config import EvalDataset
from ragbuilder.generation.utils import get_eval_dataset
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness
)
from ragas import evaluate, RunConfig
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from datetime import datetime

class Evaluator(ABC):
    @abstractmethod
    def evaluate(self, eval_dataset: Dataset) -> Dataset:
        """
        Evaluate the prompt generation Phase and returns detailed results.
        
        Returns:
        Dataset: A dataset containing the evaluation results.
        """
        pass
    def __init__(self) -> None:
        super().__init__()
        self.eval_dataset = None
class RAGASEvaluator(Evaluator):
    def __init__(self) -> None:
        super().__init__()
        print("RAGASEvaluator initiated")
    
    def evaluate(self, eval_dataset: Dataset,llm= AzureChatOpenAI(model="gpt-4o-mini"), embeddings=AzureOpenAIEmbeddings(model="text-embedding-3-large"))-> Dataset:
        result = evaluate(
                eval_dataset,
                metrics=[
                    answer_correctness,
                    faithfulness,
                    answer_relevancy,
                    context_precision,
                    context_recall,
                ],
                raise_exceptions=False, 
                is_async=True,
                run_config=RunConfig(timeout=240, max_workers=1, max_wait=180, max_retries=10)
            )
        result_df = result.to_pandas()
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_csv_path = 'rag_eval_results_'+timestamp+'.csv'
        selected_columns = ["prompt_key","prompt","question","answer","ground_truth","answer_correctness","faithfulness","answer_relevancy","context_precision","context_recall"]
        result_df[selected_columns].to_csv(output_csv_path, index=False)
        print("evaluate_prompts completed")
        print(Dataset.from_pandas(result_df[selected_columns]))
        return Dataset.from_pandas(result_df[selected_columns])

In [114]:
def _run_optimization_core(options_config: GenerationOptionsConfig):
    """
    Core function to perform RAG optimization based on the given options config.

    Args:
        options_config (GenerationOptionsConfig): Configuration for RAG optimization.

    Returns:
        Tuple: Best configuration, best score, and evaluation results.
    """
    # Load environment variables first
    # load_environment()
    # missing_vars = validate_environment(options_config)

    # if missing_vars:
    #     raise ValueError(
    #         "Missing required environment variables for selected components:\n" +
    #         "\n".join(f"- {var}" for var in missing_vars)
    #     )

    # Graph handling, if applicable
    # if options_config.graph:
    #     print("Loading graph...")
    #     config = GenerationConfig(
    #         llm_type=options_config.graph.llm.type,
    #         llm_model_kwargs=options_config.graph.llm.model_kwargs,
    #         prompt_template_path=options_config.prompt_template_path,
    #         retriever=options_config.graph.retriever,
    #     )
    #     pipeline = RAGPipeline(config)
    #     graph_chunks = pipeline.chunk_and_ingest()
        
    #     # Ensure the graph can operate with the LLM
    #     llm = options_config.graph.llm
    #     load_graph(graph_chunks, llm)

    # Create evaluator
    # if options_config.evaluator.type == EvaluatorType.CUSTOM:
    #     # module_path, class_name = options_config.evaluator.custom_class.rsplit('.', 1)
    #     module = import_module(module_path)
    #     evaluator_class = getattr(module, class_name)
    #     evaluator = evaluator_class(options_config.eval_data_set_path, options_config.evaluator)
    # else:
    #     evaluator = RAGASEvaluator(
    #         options_config.eval_data_set_path, options_config.evaluator
    #     )
    evaluator = RAGASEvaluator

    # Run optimization
    optimizer = SystemPromptGenerator(options_config, evaluator)
    optimizer.optimize()
    return
    best_config, best_score = optimizer.optimize()


    # # Create and run pipeline with best configuration for caching
    # pipeline = RAGPipeline(best_config)
    # best_index = pipeline.run()

    # # Store best configuration key in DocumentStore or equivalent
    # # optimizer.doc_store.set_best_config_key(pipeline.loader_key, pipeline.config_key)

    # console.print("[success]✓ Successfully optimized and cached best configuration[/success]")
    # return best_config, best_score, best_index

def run_optimization(options_config_path: str):
    """
    Run RAG optimization using configuration from a YAML file.

    Args:
        options_config_path (str): Path to the configuration YAML file.

    Returns:
        Tuple: Best configuration, best score, and evaluation results.
    """
    options_config = GenerationOptionsConfig.from_yaml(options_config_path)
    return _run_optimization_core(options_config)

def run_optimization_from_dict(options_config_dict: dict):
    """
    Run RAG optimization using configuration provided as a dictionary.

    Args:
        options_config_dict (dict): Configuration as a dictionary.

    Returns:
        Tuple: Best configuration, best score, and evaluation results.
    """
    options_config = GenerationOptionsConfig(**options_config_dict)
    return _run_optimization_core(options_config)


In [115]:
run_optimization('config.yaml')

RAGASEvaluator initiated
Fetching prompts from online file: https://raw.githubusercontent.com/ashwinaravind/rag_prompts/refs/heads/main/rag_prompts.yml
llms=[LLMConfig(type='openai', model_kwargs={'model_name': 'gpt-4', 'temperature': 0.7}), LLMConfig(type='azure_openai', model_kwargs={'model_name': 'azure-gpt-4', 'temperature': 0.6})] eval_data_set_path='path/to/eval_data.csv' prompt_template_path='path/to/prompt_templates.yaml' read_local_only=True evaluator='custom_evaluator' retriever='custom_retriever'
Optimization started...
type=<LLM.OPENAI: 'openai'> model_kwargs={'model_name': 'gpt-4', 'temperature': 0.7} custom_class=None


ValidationError: 1 validation error for GenerationConfig
type
  Input should be a valid string [type=string_type, input_value=LLMConfig(type=<LLM.OPENA...0.7}, custom_class=None), input_type=LLMConfig]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type